In [1]:
from pathlib import Path
import itertools
import re
import csv
import torch
import numpy as np
import scipy
import os
import nibabel as nib
import matplotlib.pyplot as plt

In [3]:
CLASS_NAMES = [ 'CN', 'AD']
DATA_DIR = '/hdd/Polygence/Data/OASIS/mri_data'
SAVE_DIR = './NumpyOASIS'

# os.mkdir(SAVE_DIR)
# for clas in CLASS_NAMES:
#     os.mkdir(SAVE_DIR+'/'+clas)

### Get filenames

In [4]:
files = list(Path(DATA_DIR).rglob("*transformed.mgz"))
print(len(files))

1022


In [5]:
image_files = {}
for key, group in itertools.groupby(sorted(files), lambda i: re.search('OAS\d+', str(i)).group()):
    image_files[key] = sorted(list(group))
print(len(image_files))

1022


In [6]:
print(len(image_files.keys()), len(set(image_files.keys())))

1022 1022


In [7]:
labels = {}
for key, filepath in image_files.items():
    label_for_this_key = None
    if 'control' in str(filepath[0]):
        label_for_this_key = 'CN'
    else:
        label_for_this_key = 'AD'
    labels[key] = label_for_this_key
print(len(labels))

1022


In [8]:
image_names = list(image_files.keys())

In [9]:
img_tst = nib.load(image_files[image_names[0]][0]).get_fdata()

In [10]:
def transforms(image):
    # image = image[slice(30, 220, 2), slice(30, 220, 2), slice(0, 220, 2)]
    
    image = np.array(image).astype(np.float32)
    
    cut=(slice(40,214,2),slice(40,190,2),slice(10,210,2))
    image = image[cut[0], cut[1], cut[2]]
    
    image = 2 * image / 255 - 1
    
    return image

image_test = transforms(img_tst)
print(image_test.shape)

(87, 75, 100)


In [11]:
print(image_test.min(), image_test.max())

-1.0 0.42561674


In [10]:
from IPython import display as dspl
import ipywidgets as widgets

# graphs the MRI image
def show_graph(img, height, width, depth):
    # Plots brain scans in all three dimensions
    fig, ax = plt.subplots(1, 3, figsize=(15,7))
    print(img.shape)

    ax[0].imshow(img[height, :, :])
    ax[1].imshow(np.rot90(img[:, width, :]))
    ax[2].imshow(np.rot90(img[:, :, depth], 3))
    

def disp(image_data):
    height, width, depth  = image_data.shape
    hi = widgets.IntSlider(value=height/2, max=height-1, continous_updates=False, description='View 1')
    wi = widgets.IntSlider(value=width/2, max=width-1, continous_updates=False, description='View 2')
    de = widgets.IntSlider(value=depth/2, max=depth-1, continous_updates=False, description='View 3')
    ui = widgets.HBox([hi, wi, de])
    
    display_output = widgets.interactive_output(show_graph, {'img':widgets.fixed(image_data),
                                                             'height':hi,
                                                             'width':wi,
                                                             'depth':de})
    dspl.display(ui, display_output)

disp(image_test)

Output()

In [11]:
CLASS_NAMES = ['CN', 'AD']

In [12]:
from tqdm.notebook import tqdm

files = 0
for index in tqdm(range(len(image_names))):
    key = image_names[index]
    
    fpath = image_files[key][0]
    
    image = nib.load(fpath).get_fdata()
    
    image = transforms(image)
    
    label = []
    for c in CLASS_NAMES:
        if c == labels[key]:
            label.append(1)
        else:
            label.append(0)
            
    print(f'Now saving image for subject {key}, sized {str(image.shape)}, with label of {label}')
    print(f'Original path was {fpath}')

    filename = SAVE_DIR+'/'+labels[key]+'/'+key+'.np'
    with open(filename, 'wb') as file:
        print(f'Saving to path {filename}')
        np.save(file, np.array([np.array([image]), np.array(label)], dtype=object))
        files += 1
        file.close()
    print('-'*60)
            
print(f'Total files: {files}')

  0%|          | 0/1022 [00:00<?, ?it/s]

Now saving image for subject OAS30001, sized (87, 75, 100), with label of [1, 0]
Original path was /hdd/Polygence/Data/OASIS/mri_data/control/OAS30001_MR_d3132/OAS30001_Freesurfer53_d3132_brain_transformed.mgz
Saving to path ./NumpyOASIS/CN/OAS30001.np
------------------------------------------------------------
Now saving image for subject OAS30002, sized (87, 75, 100), with label of [1, 0]
Original path was /hdd/Polygence/Data/OASIS/mri_data/control/OAS30002_MR_d2345/OAS30002_Freesurfer53_d2345_brain_transformed.mgz
Saving to path ./NumpyOASIS/CN/OAS30002.np
------------------------------------------------------------
Now saving image for subject OAS30003, sized (87, 75, 100), with label of [1, 0]
Original path was /hdd/Polygence/Data/OASIS/mri_data/control/OAS30003_MR_d2682/OAS30003_Freesurfer53_d2682_brain_transformed.mgz
Saving to path ./NumpyOASIS/CN/OAS30003.np
------------------------------------------------------------
Now saving image for subject OAS30004, sized (87, 75, 100)